In [1]:
# Загрузка библиотек
import pandas as pd

In [2]:
# Задание 1
# Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

# оценка 2 и ниже — низкий рейтинг;
# оценка 4 и ниже — средний рейтинг;
# оценка 4.5 и 5 — высокий рейтинг.
# Результат классификации запишите в столбец class.

t1_ratings = pd.read_csv('./hw_3/ratings.csv')
t1_ratings.head()

def rank (line): # пишем функцию
    output = ''
    if line['rating'] <= 2:
        output = 'низкий'
    elif line['rating'] <= 4:
        output = 'средний'
    else:
        output = 'высокий'
    return output

t1_ratings['rank'] = t1_ratings.apply(rank, axis=1) # применяем функцию
t1_ratings.head()

,userId,movieId,rating,timestamp,rank
0,1,31,2.5,1260759144,средний
1,1,1029,3.0,1260759179,средний
2,1,1061,3.0,1260759182,средний
3,1,1129,2.0,1260759185,низкий
4,1,1172,4.0,1260759205,средний


In [3]:
# Задание 2
# Используйте файл keywords.csv.
# Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. 
# Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. 
# Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
# Результат классификации запишите в отдельный столбец region.

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

    # NEED HELP!
    # самый простой способ получить регион по городу - перебрать словарь, но хотелось попробовать что-то поинтереснее
    # превратил словарь в DataFrame и сделал функцию, которая возвращает название региона в индекс
    # а вот что с этим можно сдеоать дальше - не разобрался
    # подскажите, как здесь лучше действовать - получать название колонки по значению или всё-таки оптимальный вариант - в лоб из словаря?
    # >>>
    # region = pd.DataFrame(geo_data)

    # def region_detector (column):
    #     return column[column == 'псков'].index

    # a = region.apply(region_detector, axis=1)
    # <<<

t2_keywords = pd.read_csv('./hw_3/keywords.csv')

    # сделал через функцию, но насколько это оптимально? можно ли сделать как-то проще/изящнее?
def region_identifier (row): # 
    # >>> создаю список из значений geo_data
    list = []
    for v in geo_data.values():
        for i in v:
            if i not in list:
                list.append(i)
    # <<<
    temp_list = row.lower().split(' ') # превращаю строку в список с разделением по пробелу
    city = "".join(set(list).intersection(set(temp_list))) # определяю, есть ли в запросе город, которые есть в списке
    # >>> поиск региона, к которому относится город по словарю
    for k, v in geo_data.items():
        if city in v:
            return k
    # <<<
    return 'unidentified'

# !РАБОТАЕТ, НО МНЕ ОЧЕНЬ НЕ НРАВИТСЯ РЕАЛИЗАЦИЯ, ПОЖАЛУЙСТА, ПРОКОММЕНТИРУЙТЕ, КАК МОЖНО СДЕЛАТЬ ЛУЧШЕ. ЗАРАНЕЕ БОЛЬШОЕ СПАСИБО!!!

t2_keywords['region'] = t2_keywords['keyword'].apply(region_identifier)
t2_keywords[t2_keywords['region'] != 'unidentified'].head(5)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


In [4]:
# Задание 3 (бонусное)
# Есть мнение, что раньше снимали настоящее кино, не то что сейчас. 
# Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. 
# Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
# Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:
    # В переменную years запишите список из всех годов с 1950 по 2010 года.
    # Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
    # Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:
        # для каждой строки пройдите по всем годам списка years;
        # если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
        # если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
        # Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.
# Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

import re #импортирую регулярные выражения

t3_ratings = pd.read_csv('./hw_3/ratings.csv') # загружаю данные
t3_movies = pd.read_csv('./hw_3/movies.csv') # загружаю данные

def prod_year (row):
    a = re.findall(r'\(\d{4}\)', str(''.join(row))) # т.к. год выпуска фильма хранится в конце записи с названием в скобках
    a = re.findall(r'\d{4}', ''.join(a)) # то достать его проще всего регулярными выражениями, также отсекая, если само название фильма также содержит год 
    a = ''.join(a) # перевожу список в строку
    if len(a) == 4: # если длина года на выходе 4 символа
        return int(a) # то ставим его и переводим строку в число
    return 1900 # если пустота, то 1900

t3_movies['pro_year'] = t3_movies['title'].apply(prod_year) # применяю функцию, добавляю столбец с годом выпуска
# объединяю датафреймы
t3_df = t3_ratings.merge(t3_movies, how = 'left', left_on=t3_ratings['movieId'], right_on=t3_movies['movieId'])
t3_df.drop(columns=['key_0', 'userId', 'movieId_y','timestamp', 'title', 'genres'], inplace=True) # убираю лишние столбцы
t3_g = t3_df.groupby(t3_df['pro_year']).agg(['count', 'mean'])['rating'].reset_index() # рассчитываю среднее
t3_g[t3_g['pro_year'] >= 1950].sort_values('mean', ascending=False).head(50) # вывожу результат
# ВЫВОД - с ростом года выпуска средний рейтинг действительно снижается однако это не может точно подтвердить предположение,
# т.к. все оценки были выставлены в последние годы, соответственно, люди смотрели и оценивали только очень хорошие старые
# фильмы, которые проверены временем. Плохие старые фильмы, возможно, в рейтинг не попали совсем



,pro_year,count,mean
44,1957,316,4.014241
59,1972,449,4.011136
39,1952,152,4.000000
41,1954,346,3.994220
38,1951,243,3.983539
61,1974,540,3.978704
49,1962,368,3.952446
37,1950,236,3.915254
64,1977,674,3.905786
51,1964,429,3.841492
